<a href="https://colab.research.google.com/github/shishir247/Assignment-with-python/blob/master/NER_using_simple_transformer_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers

In [1]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/final_data_BERT.csv",encoding="latin1" )
data.head()

,sentence_id,words,labels,POS
0,0,1,O,CD
1,0,952.wp.5977.2017.doc,O,CD
2,0,IN,B-COURTNAME,NNP
3,0,THE,I-COURTNAME,NNP
4,0,HIGH,I-COURTNAME,NNP


In [2]:
data.shape

(319723, 4)

In [3]:
data['words'] = data['words'].astype(str).str.replace('\D+', '')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [7]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [8]:
from simpletransformers.ner import NERModel,NERArgs

In [9]:
label = data["labels"].unique().tolist()
label

['O',
 'B-COURTNAME',
 'I-COURTNAME',
 'B-Act',
 'I-Act',
 'B-PETITIONER',
 'I-PETITIONER',
 'B-RESPONDENT',
 'I-RESPONDENT',
 'B-JUDGE',
 'I-JUDGE',
 'B-DATE',
 'I-DATE',
 nan]

In [12]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel ( 'bert', 'bert-base-cased', labels=label, args =args )

#model = NERModel ( 'bert',  'bert-large-cased', labels = label, args = args, use_cuda=False )

In [14]:
model.train_model ( train_data, eval_data = test_data,  acc=accuracy_score )

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(90, 0.7266440560420354)

In [15]:
result, model_outputs, preds_list = model.eval_model ( test_data )

In [16]:
result

{'eval_loss': 0.6385893755488925,
 'f1_score': 0.35542607428987616,
 'precision': 0.4250871080139373,
 'recall': 0.3053817271589487}